In [1]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
import joblib
from m4_feats_polars import *
from sklearn.metrics import mean_squared_error
from m5_sb_models import *
import polars as pl

In [2]:
# utils

def map_class(x, task, reader):
    if task.name == 'multiclass':
        return reader[x]
    else:
        return x

mapped = np.vectorize(map_class)

def score(task, y_true, y_pred):
    if task.name == 'binary':
        return roc_auc_score(y_true, y_pred)
    elif task.name == 'multiclass':
        return log_loss(y_true, y_pred)
    elif task.name == 'reg' or task.name == 'multi:reg':
        return mean_squared_error(y_true, y_pred, squared=False)
    else:
        raise 'Task is not correct.'
        
def take_pred_from_task(pred, task):
    if task.name == 'binary' or task.name == 'reg':
        return pred[:, 0]
    elif task.name == 'multiclass' or task.name == 'multi:reg':
        return pred
    else:
        raise 'Task is not correct.'
        
def use_plr(USE_PLR):
    if USE_PLR:
        return "plr"
    else:
        return "cont"

In [6]:
data_path     = 'kaggle/input/linking-writing-processes-to-writing-quality/'
train_logs    = pl.scan_csv(f'{data_path}/train_logs.csv')
test_logs    = pl.scan_csv(f'{data_path}/test_logs.csv')
train_scores = pl.scan_csv(f'{data_path}/train_scores.csv')

# PANDAS FEATS
train_essays          = get_essay_df(train_logs.collect().to_pandas())
test_essays           = get_essay_df(test_logs.collect().to_pandas())

tr_down_events_counts, ts_down_events_counts = down_events_counts(train_logs, test_logs)
tr_vect_one, ts_vect_one = countvectorize_one_one(train_essays, test_essays)
tr_pauses, ts_pauses = create_pauses(train_logs, test_logs)
tr_cursor_pos_acc, ts_cursor_pos_acc = cursor_pos_acceleration(train_logs, test_logs)
tr_r_burst, ts_r_burst = r_burst_feats(train_logs, test_logs)
tr_nuni, ts_nuni = categorical_nunique(train_logs, test_logs)
tr_e_counts_roc, ts_e_counts_roc = events_counts_rate_of_change(train_logs, test_logs, time_agg=3)
tr_wc_roc, ts_wc_roc = word_counts_rate_of_change(train_logs, test_logs)
tr_remove_pause, ts_remove_pause = remove_word_pauses(train_logs, test_logs)
tr_vect_two, ts_vect_two = countvectorize_two_one(train_essays, test_essays)
tr_word_wait, ts_word_wait = word_wait_shift(train_logs, test_logs, 1)
tr_rem_words_time_spent, ts_rem_words_time_spent = remove_words_time_spent(train_logs, test_logs)


train_feats = tr_down_events_counts.join(tr_vect_one, on='id', how='left')
train_feats = train_feats.join(tr_pauses, on='id', how='left')
train_feats = train_feats.join(tr_cursor_pos_acc, on='id', how='left')
train_feats = train_feats.join(tr_r_burst, on='id', how='left')
train_feats = train_feats.join(tr_nuni, on='id', how='left')
train_feats = train_feats.join(tr_e_counts_roc, on='id', how='left')
train_feats = train_feats.join(tr_wc_roc, on='id', how='left')
train_feats = train_feats.join(tr_remove_pause, on='id', how='left')
train_feats = train_feats.join(tr_vect_two, on='id', how='left')
train_feats = train_feats.join(tr_word_wait, on='id', how='left')
train_feats = train_feats.join(tr_rem_words_time_spent, on='id', how='left')


test_feats = ts_down_events_counts.join(ts_vect_one, on='id', how='left')
test_feats = test_feats.join(ts_pauses, on='id', how='left')
test_feats = test_feats.join(ts_cursor_pos_acc, on='id', how='left')
test_feats = test_feats.join(ts_r_burst, on='id', how='left')
test_feats = test_feats.join(ts_nuni, on='id', how='left')
test_feats = test_feats.join(ts_e_counts_roc, on='id', how='left')
test_feats = test_feats.join(ts_wc_roc, on='id', how='left')
test_feats = test_feats.join(ts_remove_pause, on='id', how='left')
test_feats = test_feats.join(ts_vect_two, on='id', how='left')
test_feats = test_feats.join(ts_word_wait, on='id', how='left')
test_feats = test_feats.join(ts_rem_words_time_spent, on='id', how='left')

train_logs = train_logs.collect().to_pandas()
test_logs = test_logs.collect().to_pandas()
train_scores = train_scores.collect().to_pandas()
train_feats = train_feats.collect().to_pandas()
test_feats = test_feats.collect().to_pandas()

train_feats           = train_feats.merge(parag_feats(train_essays), on='id', how='left')
test_feats            = test_feats.merge(parag_feats(test_essays), on='id', how='left')
train_feats           = train_feats.merge(sent_feats(train_essays), on='id', how='left')
test_feats            = test_feats.merge(sent_feats(test_essays), on='id', how='left')
train_feats           = train_feats.merge(word_feats(train_essays), on='id', how='left')
test_feats            = test_feats.merge(word_feats(test_essays), on='id', how='left')

tr_ids = train_feats.id
ts_ids = test_feats.id

feats = pd.concat([train_feats,test_feats], axis=0)
feats = preprocess_feats(feats)
train_feats = feats[feats['id'].isin(tr_ids)]
test_feats = feats[feats['id'].isin(ts_ids)]

train_feats           = train_feats.merge(train_scores, on=['id'], how='left')
print(f'train feats shape {train_feats.shape}')

< Events counts features >
< Count vectorize one-grams >
< Idle time features >
< cursor position acceleration >
< R-burst features >
< Categorical # unique values features >
< event_id rate of change >
< Word counts rate of change features >
< removed words pauses basic
< Count vectorize bi-grams >
< word_wait_shift >
< remove_words_time_spent >
< Essays paragraphs feats >
< Essays paragraphs feats >
< Essays sentences feats >
< Essays sentences feats >
< Essays word feats >
< Essays word feats >
train feats shape (2471, 186)


In [8]:
import torch

N_THREADS = 16
N_FOLDS = 10
RANDOM_STATE = 42
TEST_SIZE = 0.2
VAL_SIZE = 0.2
TIMEOUT = 10000
ADVANCED_ROLES = False
USE_QNT = True
TASK = 'reg'
ALGOS_FOR_BLEND =  ["dense", "tabnet", "fttransformer", "autoint"]
USE_PLR = True
TRAIN_BS = 128
USE_FS = False
FEATURE_RATIO = 0.8

# example for binary classification
TARGET_NAME = 'score'
    
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

task = Task(TASK)

# example for binary classification
roles = {
    'target': TARGET_NAME,
    #'drop': ['id']
}

In [9]:
algo = 'denselight'
automl = TabularAutoML(
    task = task, 
    timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    general_params = {"use_algos": [[algo]]}, # ['nn', 'mlp', 'dense', 'denselight', 'resnet', 'snn', 'node', 'autoint', 'fttransformer'] or custom torch model
    nn_params = {
        "n_epochs": 100, 
        "bs": TRAIN_BS, 
        "num_workers": 0, 
        "path_to_save": None, 
        "freeze_defaults": True,
       # "cont_embedder": use_plr(USE_PLR),
    },
    nn_pipeline_params = {"use_qnt": USE_QNT, "use_te": False},
    reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE, 'advanced_roles': ADVANCED_ROLES},
)

oof_pred = automl.fit_predict(train_feats, roles = roles, verbose = 3)
test_pred = automl.predict(test_feats)
oof = score(task, mapped(train_feats[TARGET_NAME].values, task, automl.reader.class_mapping), take_pred_from_task(oof_pred.data, task))
denselight_list = [(oof, oof_pred.data[:, 0], test_pred.data[:, 0])]

[11:58:31] Stdout logging level is INFO3.
[11:58:31] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[11:58:31] Task: reg

[11:58:31] Start automl preset with listed constraints:
[11:58:31] - time: 10000.00 seconds
[11:58:31] - CPU: 16 cores
[11:58:31] - memory: 16 GB

[11:58:31] Train data shape: (2471, 186)

[11:58:31] Layer 1 train process start. Time left 9999.93 secs
[11:58:32] Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN_denselight_0 ...
[11:58:32] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_TorchNN_denselight_0 =====
[11:58:35] Epoch: 0, train loss: 0.7018664479255676, val loss: 0.40319305658340454, val metric: -0.40131697058677673
[11:58:35] Epoch: 1, train loss: 0.41542747616767883, val loss: 0.3703724145889282, val metric: -0.36911892890930176
[11:58:35] Epoch: 2, train loss: 0.3619755506515503, val loss: 0.3633615970611572, val metric: -0.36206766963005066
[11:58:35] Epoch: 3, train loss: 0.3239513337612152, val loss:

3